In [2]:
from datasets import load_dataset

dataset = load_dataset("1aurent/PatchCamelyon")

# Optional: subset to first 10,000 samples
dataset = dataset["train"].shuffle(seed=42).select(range(10000))

# Inspect
example = dataset[0]
image = example['image']

image.save("sample_image.png")
print("Image saved to sample_image.png")

print(dataset)
print(dataset.features)

Image saved to sample_image.png
Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})
{'image': Image(mode=None, decode=True, id=None), 'label': Value(dtype='bool', id=None)}


# File Structure

```text
patchcamelyon_subset/
├── tumor/
│   ├── img00001.png
│   ├── img00002.png
│   └── ...
└── normal/
    ├── img00001.png
    ├── img00002.png
    └── ...
```

In [ ]:
example["messages"] = [
        {"role": "user","content": [{"type": "image",},{"type": "text","text": PROMPT,},],},
        {
            "role": "assistant",
            "content": [
                {
                    "type": "text",
                    "text": HISTOPATHOLOGY_CLASSES[example["label"]],
                },
            ],
        },
    ]

In [10]:
from typing import Any
a: dict[str, Any] = {}
a['messages'] = [{1,2},{'hello', 2}]
print(a['messages'])


[{1, 2}, {2, 'hello'}]
